In [1]:
from utils import *
import numpy as np
import torch, os
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
set_random_seed(3)

Seed: 3


In [3]:
# def scaling_embeddings(embeddings):
#     for i in range(len(embeddings)):
#         embeddings[i] = StandardScaler().fit_transform(embeddings[i])
#     return embeddings

def reshape_embeddings(embeddings):
    if embeddings.shape[1] == 1:
        embeddings = embeddings.squeeze(1)
    return embeddings

In [4]:
dynahate_labels_train = process_labels(read_labels("dynahate", "train"))
dynahate_labels_dev = process_labels(read_labels("dynahate", "dev"))
dynahate_labels_test = process_labels(read_labels("dynahate", "test"))

latenthatred_labels_train = read_labels("latenthatred", "train")
latenthatred_labels_dev = read_labels("latenthatred", "dev")
latenthatred_labels_test = read_labels("latenthatred", "test")

olid_labels_train = read_labels("olid", "train")
olid_labels_dev = read_labels("olid", "dev")
olid_labels_test = read_labels("olid", "test")

In [5]:
mlp = MLPClassifier(random_state=3)
gridsearch = GridSearchCV(
    mlp,
    param_grid={
        "hidden_layer_sizes": [(128), (128,64)],
        "activation": ["relu"],
        "solver": ["adam"],
        "learning_rate_init": [0.001, 0.0001],
        "learning_rate": ["adaptive"],
        "early_stopping": [True],
        "max_iter": [10000],
    },
    verbose=4,
    n_jobs=os.cpu_count()//3,
    cv=5,
)

In [6]:
dynahate_labels_train_dev = np.concatenate((dynahate_labels_train, dynahate_labels_dev))
latenthatred_labels_train_dev = np.concatenate((latenthatred_labels_train, latenthatred_labels_dev))
olid_labels_train_dev = np.concatenate((olid_labels_train, olid_labels_dev))

In [7]:
bert_dynahate_train_embeddings = reshape_embeddings(bert_dynahate_train_embeddings)
bert_dynahate_dev_embeddings = reshape_embeddings(bert_dynahate_dev_embeddings)
bert_dynahate_test_embeddings = reshape_embeddings(bert_dynahate_test_embeddings)

bert_latenthatred_train_embeddings = reshape_embeddings(bert_latenthatred_train_embeddings)
bert_latenthatred_dev_embeddings = reshape_embeddings(bert_latenthatred_dev_embeddings)
bert_latenthatred_test_embeddings = reshape_embeddings(bert_latenthatred_test_embeddings)

bert_olid_train_embeddings = reshape_embeddings(bert_olid_train_embeddings)
bert_olid_dev_embeddings = reshape_embeddings(bert_olid_dev_embeddings)
bert_olid_test_embeddings = reshape_embeddings(bert_olid_test_embeddings)

### BERT

#### DynaHate

In [8]:
bert_dynahate_train_dev = np.concatenate((bert_dynahate_train_embeddings, bert_dynahate_dev_embeddings))

grid_results = gridsearch.fit(bert_dynahate_train_dev, dynahate_labels_train_dev)

best_params = grid_results.best_params_
mlp = grid_results.best_estimator_

print("Best params: ", best_params)

train_dev_preds = mlp.predict(bert_dynahate_train_dev)
test_preds = mlp.predict(bert_dynahate_test_embeddings)

computeAllScores(train_dev_preds, test_preds, dynahate_labels_train_dev, dynahate_labels_test, "Results/bert_dynahate_seed_3")
save_model(mlp, "Saves/bert_dynahate_seed_3")

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best params:  {'activation': 'relu', 'early_stopping': True, 'hidden_layer_sizes': 128, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'max_iter': 10000, 'solver': 'adam'}
Accuracy Train Dev:  0.681233794295592
Accuracy Test:  0.6677184466019418
Weighted F1 Train Dev:  0.6767145258568324
Weighted F1 Test:  0.6616896552466864
Macro F1 Train Dev:  0.672742012011798
Macro F1 Test:  0.6550017160365899
Micro F1 Train Dev:  0.681233794295592
Micro F1 Test:  0.6677184466019418
Weighted Recall Train Dev:  0.681233794295592
Weighted Recall Test:  0.6677184466019418
Macro Recall Train Dev:  0.672916236993226
Macro Recall Test:  0.6550109324587366
Micro Recall Train Dev:  0.681233794295592
Micro Recall Test:  0.6677184466019418
Confusion Matrix Train Dev: 
[[ 9629  7488]
 [ 4314 15593]]
Confusion Matrix Test: 
[[ 980  872]
 [ 497 1771]]


#### LatentHatred

In [9]:
bert_latenthatred_train_dev = np.concatenate((bert_latenthatred_train_embeddings, bert_latenthatred_dev_embeddings))

grid_results = gridsearch.fit(bert_latenthatred_train_dev, latenthatred_labels_train_dev)

best_params = grid_results.best_params_
mlp = grid_results.best_estimator_

print("Best params: ", best_params)

train_dev_preds = mlp.predict(bert_latenthatred_train_dev)
test_preds = mlp.predict(bert_latenthatred_test_embeddings)

computeAllScores(train_dev_preds, test_preds, latenthatred_labels_train_dev, latenthatred_labels_test, "Results/bert_latenthatred_seed_3")
save_model(mlp, "Saves/bert_latenthatred_seed_3")

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best params:  {'activation': 'relu', 'early_stopping': True, 'hidden_layer_sizes': (128, 64), 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'max_iter': 10000, 'solver': 'adam'}
Accuracy Train Dev:  0.6975170701427684
Accuracy Test:  0.6869646182495345
Weighted F1 Train Dev:  0.658462600299731
Weighted F1 Test:  0.6470074870648039
Macro F1 Train Dev:  0.44477925086175857
Macro F1 Test:  0.4296343487138765
Micro F1 Train Dev:  0.6975170701427684
Micro F1 Test:  0.6869646182495345
Weighted Recall Train Dev:  0.6975170701427684
Weighted Recall Test:  0.6869646182495345
Macro Recall Train Dev:  0.4441840234187908
Macro Recall Test:  0.4316370330282055
Micro Recall Train Dev:  0.6975170701427684
Micro Recall Test:  0.6869646182495345
Confusion Matrix Train Dev: 
[[9133  805    7]
 [3254 2084    9]
 [ 458  340   20]]
Confusion Matrix Test: 
[[3038  306    2]
 [1104  646    3]
 [ 147  119    5]]


#### OLID

In [10]:
bert_olid_train_dev = np.concatenate((bert_olid_train_embeddings, bert_olid_dev_embeddings))

grid_results = gridsearch.fit(bert_olid_train_dev, olid_labels_train_dev)

best_params = grid_results.best_params_
mlp = grid_results.best_estimator_

print("Best params: ", best_params)

train_dev_preds = mlp.predict(bert_olid_train_dev)
test_preds = mlp.predict(bert_olid_test_embeddings)

computeAllScores(train_dev_preds, test_preds, olid_labels_train_dev, olid_labels_test, "Results/bert_olid_seed_3")

save_model(mlp, "Saves/bert_olid_seed_3")

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best params:  {'activation': 'relu', 'early_stopping': True, 'hidden_layer_sizes': 128, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'max_iter': 10000, 'solver': 'adam'}
Accuracy Train Dev:  0.7532477341389728
Accuracy Test:  0.7941860465116279
Weighted F1 Train Dev:  0.7419993190953723
Weighted F1 Test:  0.7722668660167745
Macro F1 Train Dev:  0.6992765108887113
Macro F1 Test:  0.6958513743833408
Micro F1 Train Dev:  0.7532477341389727
Micro F1 Test:  0.7941860465116279
Weighted Recall Train Dev:  0.7532477341389728
Weighted Recall Test:  0.7941860465116279
Macro Recall Train Dev:  0.688621966269025
Macro Recall Test:  0.6746639784946237
Micro Recall Train Dev:  0.7532477341389728
Micro Recall Test:  0.7941860465116279
Confusion Matrix Train Dev: 
[[7791 1049]
 [2218 2182]]
Confusion Matrix Test: 
[[586  34]
 [143  97]]
